In [1]:
!pip install sentence-transformers

In [2]:
# Importing packages
import io
import pandas as pd
import datetime as dt
import numpy as np
import dateutil
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sentence_transformers import SentenceTransformer, util
import re


from plotly.subplots import make_subplots

2025-09-10 05:34:39.700265: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [4]:
claims_raw = pd.read_excel("Claims Portfolio_July2024toJuly2025.xlsx")

In [5]:
claims_raw.shape

(5224, 19)

In [6]:
claims_raw.columns

Index(['Assigned User', 'Status', 'Completed', 'Claim Number', 'e5 Reference',
       'Work Type', 'Process Type', 'Policy Number', 'Client', 'Customer Name',
       'Date Received', 'Absolute Age', 'Time in current status',
       'Current Estimate', 'Current Payment', 'Check Existing Claims',
       'Customer Care Case', 'Claim Description', 'Indemnity Decision'],
      dtype='object')

In [7]:
# Feature Engineering

In [8]:
# Extract specific columns
df_claims_desc = claims_raw[['Absolute Age','Claim Description','Indemnity Decision']]


# Step 1: Create bins every 10 years up to max age + 10
bins = list(range(0, df_claims_desc['Absolute Age'].max() + 11, 10))

# Step 2: Create labels for bins (e.g., "0-9", "10-19", ...)
labels = [f"{bins[i]}-{bins[i+1]-1}" for i in range(len(bins)-1)]

# Step 3: Apply pd.cut
df_claims_desc['DaysBin'] = pd.cut(
    df_claims_desc['Absolute Age'],
    bins=bins,
    labels=labels,
    right=False,          # left-inclusive, so 0–9, 10–19, etc.
    include_lowest=True
)

# Save DataFrame to CSV
df_claims_desc.to_csv("claims_desc_input.csv", index=False)

In [9]:

# Remove names from the description and replacing the null values

def clean_text(text):
    """Regex + spaCy name removal, handles null values"""
    if not isinstance(text, str) or pd.isna(text):
        return ""  
    
    # Regex cleanup
    # Full names (Firstname Lastname)
    text = re.sub(r"\b[A-Z][a-z]+ [A-Z][a-z]+\b", "", text)

    # Single capitalized word before a dash (Christopher - Fr)
    text = re.sub(r"\b[A-Z][a-z]+(?=\s*-\s*)", "", text)

    # Possessive names (Adrian's, John's)
    text = re.sub(r"\b[A-Z][a-z]+(?='s)\b", "", text)
    
    return " ".join(text.split())

def process_and_save(input_csv, output_csv, chunk_size=1000):
    """Read, process, and write cleaned data in chunks"""
    reader = pd.read_csv(input_csv, chunksize=chunk_size)
    
    for i, chunk in enumerate(reader):
        print(f"🔄 Processing chunk {i+1}...")
        chunk["Description"] = chunk["Claim Description"].apply(clean_text)
        
        # Append to file (mode='a' for append after first chunk)
        if i == 0:
            chunk.to_csv(output_csv, index=False, mode="w")  # overwrite/create
        else:
            chunk.to_csv(output_csv, index=False, header=False, mode="a")  # append
    
    print(f"✅ Finished! Cleaned file saved at {output_csv}")


In [10]:
# Read the CSV in chunks of 1000 rows
chunk_size = 1000
for i, chunk in enumerate(pd.read_csv("claims_desc_input.csv", chunksize=chunk_size)):
    print(f"🔄 Processing chunk {i+1} with {len(chunk)} rows...")

process_and_save("claims_desc_input.csv", "claims_cleaned.csv", chunk_size=1000)

🔄 Processing chunk 1 with 1000 rows...
🔄 Processing chunk 2 with 1000 rows...
🔄 Processing chunk 3 with 1000 rows...
🔄 Processing chunk 4 with 1000 rows...
🔄 Processing chunk 5 with 1000 rows...
🔄 Processing chunk 6 with 224 rows...
🔄 Processing chunk 1...
🔄 Processing chunk 2...
🔄 Processing chunk 3...
🔄 Processing chunk 4...
🔄 Processing chunk 5...
🔄 Processing chunk 6...
✅ Finished! Cleaned file saved at claims_cleaned.csv


In [11]:
df_claims_cleaned = pd.read_csv("claims_cleaned.csv")
print(df_claims_cleaned.head())

   Absolute Age                                  Claim Description  \
0            92                      Sinisa - claim due to illness   
1             6  Claire - diagnosed with acute diarrhea and mil...   
2            26  Adrienne's family fell violently ill, dehydration   
3            65                             Eddie- stolen bagpack    
4            10          Thi Bich Lien Nguyen - abdo pain_x000D_\n   

  Indemnity Decision DaysBin  \
0                NaN   90-99   
1                NaN     0-9   
2                NaN   20-29   
3                NaN   60-69   
4                NaN   10-19   

                                         Description  
0                             - claim due to illness  
1  - diagnosed with acute diarrhea and mild dehyd...  
2          's family fell violently ill, dehydration  
3                                   - stolen bagpack  
4                                 - abdo pain_x000D_  


In [12]:
# Display the result
# Show entire content of each cell
pd.set_option("display.max_colwidth", None)

# Show all rows
pd.set_option("display.max_rows", None)

for claim, desc in zip(df_claims_cleaned["Claim Description"], df_claims_cleaned["Description"]):
    print(f"Claim Description: {claim}\nDescription: {desc}\n{'-'*80}")

Claim Description: Sinisa - claim due to illness
Description: - claim due to illness
--------------------------------------------------------------------------------
Claim Description: Claire - diagnosed with acute diarrhea and mild dehydration
Description: - diagnosed with acute diarrhea and mild dehydration
--------------------------------------------------------------------------------
Claim Description: Adrienne's family fell violently ill, dehydration
Description: 's family fell violently ill, dehydration
--------------------------------------------------------------------------------
Claim Description: Eddie- stolen bagpack 
Description: - stolen bagpack
--------------------------------------------------------------------------------
Claim Description: Thi Bich Lien Nguyen - abdo pain_x000D_

Description: - abdo pain_x000D_
--------------------------------------------------------------------------------
Claim Description: Taya - snowboarding accident, injured left foot
Descriptio

In [13]:
import torch

# -----------------------------
# 1. Define categories with descriptive guidance (based on rules)
# -----------------------------
categories = {
    "Medical": "medication,telehealth, Illness, injury, hospital, doctor, treatment, surgery, medicine, fever, infection, fracture, burn, pain, Bali Belly, gastro",
    "Emergency Dental": "Tooth, dentist, dental treatment, teeth problem",
    "COVID-19 Overseas Medical": "Covid, coronavirus, positive test, PCR, quarantine, pandemic, covid sickness",
    "Luggage Cover": "Lost luggage, stolen bag, theft, stolen passport, damaged bag, missing luggage, pickpocket, backpack, baggage",
    "Special Events": "Missed wedding, funeral, graduation, event delayed, special event, sucide",
    "Accommodation or Transport Expenses": "onboarding, flight, Unexpected accommodation, hotel stay, transport cost, taxi, extra flight, overnight stay",
    "Cancellation Cover": "canx, Trip cancellation, trip amendment, prepaid non-refundable, abandoned trip, resumption, return to Australia",
    "Personal Liability": "Legally liable, compensation, damage to property, injury to others, third party claim, negligence",
    "Rental Vehicle Insurance": "Rental car, hire car, vehicle, rental accident, damaged rental car, moped rental",
    "Adventure Activities": "Scuba diving, sky diving, rock climbing, rafting, bungee, zipline, paragliding",
    "Motorcycle Cover": "Motorbike, moped, scooter, motorcycle, bike accident",
    "Cruise Cover": "Cruise, ship, cabin, onboard doctor, port, sea",
    "Snow Sports": "Skiing, snowboarding, snowmobile, heli-skiing, ice skating, snow accident",
    "Gadget Cover": "Mobile, camera, laptop, drone, electronics, video cameras, gopro, tablet, smart watches",
    "Other": "pls see, duplicate"
}

# -----------------------------
# 2. Load SentenceTransformer for classification
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# categories = { "CategoryName": "Category Description", ... }  # define your categories
cat_names = list(categories.keys())
cat_embeddings = model.encode(list(categories.values()), convert_to_tensor=True, device=device)

# -----------------------------
# 3. Encode all claims in batches
# -----------------------------
claims = df_claims_cleaned["Description"].astype(str).tolist()
claim_embeddings = model.encode(claims, convert_to_tensor=True, batch_size=64, device=device)

# -----------------------------
# 4. Compute cosine similarities and assign best category
# -----------------------------
cos_scores = util.cos_sim(claim_embeddings, cat_embeddings) 
best_indices = torch.argmax(cos_scores, dim=1).cpu().numpy()
df_claims_cleaned["Category"] = [cat_names[i] for i in best_indices]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/82 [00:00<?, ?it/s]

In [14]:
# 6. Save to new Excel
output_path = "Claims_Portfolio_Categorised.xlsx"
df_claims_cleaned.to_excel(output_path, index=False)

print(f"✅ Classification complete. Saved to {output_path}")

✅ Classification complete. Saved to Claims_Portfolio_Categorised.xlsx


In [15]:
subset = df_claims_cleaned[df_claims_cleaned["Category"] == "Other"]

from IPython.display import display

display(subset[["Description"]])


,Description
25,- had Impetigo crustosa
35,- Glaucoma
84,- Cough_x000D_
285,NaN
286,NaN
310,- - EMC Decline_x000D_
317,Please refer to 1588602_x000D_
439,NaN
456,NaN
463,NaN


In [16]:
# Visualize the category field

import plotly.express as px
import plotly.graph_objects as go

# Count records per category
category_counts = df_claims_cleaned["Category"].value_counts().reset_index()
category_counts.columns = ["Category", "Count"]

# Compute percentage
category_counts["Percent"] = (category_counts["Count"] / category_counts["Count"].sum()) * 100

# Create initial bar chart (default: Count)
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=category_counts["Category"],
        y=category_counts["Count"],
        text=category_counts["Count"],
        textposition="outside",
        marker_color=px.colors.qualitative.Set3,
        name="Count"
    )
)

# Add dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    label="Count",
                    method="update",
                    args=[{"y": [category_counts["Count"]],
                           "text": [category_counts["Count"]]},
                          {"yaxis": {"title": "Number of Claims"}}]
                ),
                dict(
                    label="Percentage",
                    method="update",
                    args=[{"y": [category_counts["Percent"]],
                           "text": [category_counts["Percent"].round(1).astype(str) + '%']},
                          {"yaxis": {"title": "Percentage of Claims"}}]
                )
            ]),
            direction="down",
            x=0.5, xanchor="center",
            y=1.15, yanchor="top"
        )
    ]
)

# General layout
fig.update_layout(
    title="Number of Claims per Category",
    xaxis_title="Category",
    yaxis_title="Number of Claims",
    xaxis_tickangle=-45,
    showlegend=False,
    height=700  # Increased height
)

fig.show()
